In [14]:
import os
from pathlib import Path
from dotenv import load_dotenv

# Load .env file from the same directory as this notebook
# In Jupyter, the current working directory is typically where the notebook is located
try:
    env_file = Path('.env')
    if env_file.exists():
        load_dotenv(dotenv_path=env_file, override=True)
        google_api_key = os.getenv("GOOGLE_API_KEY")
        if not google_api_key:
            raise ValueError("GOOGLE_API_KEY not found in .env file")
        print(f"✅ Gemini API key loaded from .env file (first 10 chars: {google_api_key[:10]}...)")
        print(f"✅ Verified: GOOGLE_API_KEY is set in environment (length: {len(google_api_key)} chars)")
        print("ℹ️  ADK will use GOOGLE_API_KEY")
except Exception as e:
    print(f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your .env file. Details: {e}")

✅ Gemini API key loaded from .env file (first 10 chars: AIzaSyCxBj...)
✅ Verified: GOOGLE_API_KEY is set in environment (length: 39 chars)
ℹ️  ADK will use GOOGLE_API_KEY


In [15]:
from typing import Any, Dict

from google.adk.agents import Agent, LlmAgent
from google.adk.apps.app import App, EventsCompactionConfig
from google.adk.models.google_llm import Gemini
from google.adk.sessions import DatabaseSessionService
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.adk.tools.tool_context import ToolContext
from google.genai import types

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


In [16]:
MODEL_NAME= "gemini-2.5-flash-lite"
APP_NAME = "personal-fitness-trainer-tracker"
async def run_session(runner : Runner, user_id: str, session_id: str, user_queries: list[str] | str = None):
    app = runner.app_name
    try:
        session = await session_service.create_session(
            app_name=app, user_id=user_id, session_id=session_id
        )
    except:
        session = await session_service.get_session(
            app_name=app, user_id=user_id, session_id=session_id
        )
    
    if user_queries:
        if type(user_queries) == str:
            user_queries = [user_queries]

        for query in user_queries:
            print(f"\nUser > {query}")
            query = types.Content(role="user", parts=[types.Part(text=query)])
            async for event in runner.run_async(
                user_id=user_id, session_id=session.id, new_message=query
            ):
                if event.content and event.content.parts:
                    # Filter out empty or "None" responses before printing
                    if (
                        event.content.parts[0].text != "None"
                        and event.content.parts[0].text
                    ):
                        print(f"{MODEL_NAME} > ", event.content.parts[0].text)
    else:
        print("No queries!")

In [17]:
retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

In [ ]:
personalised_fitness_trainer_agent = LlmAgent(
    model=Gemini(model_name=MODEL_NAME, retry_options=retry_config),
    name="personalised_fitness_trainer_agent",
    description="A helpful assistant that can answer questions and help with tasks.",
)

db_url = "sqlite:///client_data.db"
session_service = DatabaseSessionService(db_url=db_url)
runner = Runner(agent=personalised_fitness_trainer_agent, app_name=APP_NAME, session_service=session_service)

print("✅ Upgraded to persistent sessions!")
print(f"   - Database: my_agent_data.db")
print(f"   - Sessions will survive restarts!")

✅ Upgraded to persistent sessions!
   - Database: my_agent_data.db
   - Sessions will survive restarts!


In [20]:
await run_session(
    runner,
    "ayoush",
    "test-db-session-01",
    ["Hi, I am Ayoush! In Day 1, I did cardio and squats", "Hello! What I did in Day 1 workout can you tell me?"],
)

Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.



User > Hi, I am Ayoush! In Day 1, I did cardio and squats
gemini-2.5-flash-lite >  Hi Ayoush! Thanks for sharing your Day 1 workout – cardio and squats sound like a great start.

How can I help you with your fitness journey? Are you looking for:
*   Help planning future workouts?
*   Nutrition advice?
*   Tips on form or technique?
*   Motivation and tracking?
*   Something else entirely?

User > Hello! What I did in Day 1 workout can you tell me?
gemini-2.5-flash-lite >  Hi Ayoush!

On **Day 1**, you did:
*   **Cardio**
*   **Squats**


In [21]:
await run_session(
    runner,
    "ayoush",
    "test-db-session-01",
    ["What's my name? And what I did on day 1?", "Can you tell me some of the chest workouts I can do in day 2?", "I did back exercise in Day 3"],
)


User > What's my name? And what I did on day 1?
gemini-2.5-flash-lite >  Your name is **Ayoush**.

On Day 1, you did **cardio** and **squats**.

User > Can you tell me some of the chest workouts I can do in day 2?
gemini-2.5-flash-lite >  Certainly, Ayoush! For Day 2, here are some effective chest workouts you can consider, depending on what equipment you have available:

**1. Push-ups (Bodyweight)**
*   **How to:** Start in a plank position with your hands slightly wider than shoulder-width apart, fingers pointing forward. Lower your chest towards the floor by bending your elbows, keeping your body in a straight line from head to heels. Push back up to the starting position.
*   **Variations:**
    *   **Knee push-ups:** If regular push-ups are too challenging.
    *   **Incline push-ups:** Hands on an elevated surface (bench, sturdy chair) to make it easier.
    *   **Decline push-ups:** Feet elevated on a bench to make it harder and target the upper chest more.

**2. Dumbbell Press

In [22]:
await run_session(
    runner, "komal", "test-db-session-02",["Hello! What is my name?"], 
)  # Note, we are using new session name


User > Hello! What is my name?
gemini-2.5-flash-lite >  I'm sorry, I don't know your name. As an AI, I don't have access to personal information about you from past conversations or any other source.


In [23]:
import sqlite3

def check_data_in_db():
    with sqlite3.connect("client_data.db") as connection:
        cursor = connection.cursor()
        result = cursor.execute(
            "select app_name, session_id, author, content from events"
        )
        print([_[0] for _ in result.description])
        for each in result.fetchall():
            print(each)


check_data_in_db()

['app_name', 'session_id', 'author', 'content']
('personal-fitness-trainer-tracker', 'test-db-session-01', 'user', '{"parts": [{"text": "Hi, I am Ayoush! In Day 1, I did cardio and squats"}], "role": "user"}')
('personal-fitness-trainer-tracker', 'test-db-session-01', 'personalised_fitness_trainer_agent', '{"parts": [{"text": "Hi Ayoush! Thanks for sharing your Day 1 workout \\u2013 cardio and squats sound like a great start.\\n\\nHow can I help you with your fitness journey? Are you looking for:\\n*   Help planning future workouts?\\n*   Nutrition advice?\\n*   Tips on form or technique?\\n*   Motivation and tracking?\\n*   Something else entirely?"}], "role": "model"}')
('personal-fitness-trainer-tracker', 'test-db-session-01', 'user', '{"parts": [{"text": "Hello! What I did in Day 1 workout can you tell me?"}], "role": "user"}')
('personal-fitness-trainer-tracker', 'test-db-session-01', 'personalised_fitness_trainer_agent', '{"parts": [{"text": "Hi Ayoush!\\n\\nOn **Day 1**, you did

In [24]:
personalised_fitness_app = App(
    name="personalised_fitness_app",
    root_agent=personalised_fitness_trainer_agent,
    events_compaction_config=EventsCompactionConfig(
        compaction_interval=3,  # Trigger compaction every 3 invocations
        overlap_size=1,  # Keep 1 previous turn for context
    ))

personalised_fitness_runner = Runner(
    app=personalised_fitness_app, session_service=session_service
)



/var/folders/m5/vw686v2x6xj9k9z7211qm4jh0000gn/T/ipykernel_57908/419681120.py:4: UserWarning: [EXPERIMENTAL] EventsCompactionConfig: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  events_compaction_config=EventsCompactionConfig(


In [25]:
# Turn 1
await run_session(
    personalised_fitness_runner,
    "Ayoush",
    "compaction_demo",
    "What is the latest news about AI in healthcare?",
)

# Turn 2
await run_session(
    personalised_fitness_runner,
    "Ayoush",
    "compaction_demo",
    "Are there any new developments in drug discovery?",
)

# Turn 3 - Compaction should trigger after this turn!
await run_session(
    personalised_fitness_runner,
    "Ayoush",
    "compaction_demo",
    "Tell me more about the second development you found.",
)

# Turn 4
await run_session(
    personalised_fitness_runner,
    "Ayoush",
    "compaction_demo",
    "Who are the main companies involved in that?",
)


User > What is the latest news about AI in healthcare?
gemini-2.5-flash-lite >  As an AI, I don't have real-time access to breaking news beyond my last training update. Therefore, I can't give you the absolute latest headlines from this minute or day.

However, I can tell you that AI in healthcare is a rapidly evolving field, and there's constant innovation. To get the very latest news, I'd recommend checking reputable sources like:

*   **Medical Journals & Publications:** *Nature Medicine*, *The Lancet Digital Health*, *JAMA Network Open*, *New England Journal of Medicine (NEJM) AI*
*   **Tech & Science News Outlets:** *MIT Technology Review*, *STAT News*, *Wired*, *TechCrunch (for startup news)*, *Fierce Healthcare AI*
*   **Industry Blogs & Conferences:** Look for summaries from recent AI in healthcare conferences (e.g., HIMSS, RSNA AI Showcase, NVIDIA GTC healthcare tracks).

When you look for news, you'll likely find articles on topics such as:

*   **Drug Discovery and Developm

In [26]:
# Get the final session state
final_session = await session_service.get_session(
    app_name=personalised_fitness_runner.app_name,
    user_id="Ayoush",
    session_id="compaction_demo",
)

print("--- Searching for Compaction Summary Event ---")
found_summary = False
for event in final_session.events:
    # Compaction events have a 'compaction' attribute
    if event.actions and event.actions.compaction:
        print("\n✅ SUCCESS! Found the Compaction Event:")
        print(f"  Author: {event.author}")
        print(f"\n Compacted information: {event}")
        found_summary = True
        break

if not found_summary:
    print(
        "\n❌ No compaction event found. Try increasing the number of turns in the demo."
    )

--- Searching for Compaction Summary Event ---

✅ SUCCESS! Found the Compaction Event:
  Author: user

 Compacted information: model_version=None content=None grounding_metadata=None partial=None turn_complete=None finish_reason=None error_code=None error_message=None interrupted=None custom_metadata=None usage_metadata=None live_session_resumption_update=None input_transcription=None output_transcription=None avg_logprobs=None logprobs_result=None cache_metadata=None citation_metadata=None invocation_id='8a3fe2c6-1067-48fb-8115-bfaedea98921' author='user' actions=EventActions(skip_summarization=None, state_delta={}, artifact_delta={}, transfer_to_agent=None, escalate=None, requested_auth_configs={}, requested_tool_confirmations={}, compaction={'start_timestamp': 1763439611.539153, 'end_timestamp': 1763439622.243111, 'compacted_content': {'parts': [{'function_call': None, 'code_execution_result': None, 'executable_code': None, 'file_data': None, 'function_response': None, 'inline_data'